In [4]:
import math
import itertools

import matplotlib as plt
import pandas as pd
import numpy as np

#needed to model temps through year from monthly averages
from scipy.interpolate import *


%cd "/Users/laurenneel/Desktop"


Ta_input = pd.DataFrame(pd.read_csv("Tempe_microclims.csv"))
Ta_input.columns = ['', 'dates', 'DOY', 'TIME','Ta_sun','Ta_shade'] #column headers that you want to use, doesn't have to match csv file
#Ta_input.to_numpy()

#Ta_input[['dates','Ta_sun']] #print only certain columns
#Ta_input['Ta_sun'].max() # to print max of a column
#Ta_input.describe()

dates=list(Ta_input['dates'])
DOY =list(Ta_input['DOY'])
TIME=list(Ta_input['TIME'])
Ta_sun=list(Ta_input['Ta_sun'])
Ta_shade=list(Ta_input['Ta_shade'])



micro_df = pd.DataFrame(list(zip(dates,DOY,TIME,Ta_sun,Ta_shade)),
                columns=['dates','DOY', 'TIME','Ta_sun','Ta_shade'])


print(micro_df)


#plt.plot(Ta_input.dates, Ta_input.Ta_sun)
#plt.plot(Ta_input.dates, Ta_input.Ta_shade)

/Users/laurenneel/Desktop
           dates  DOY  TIME    Ta_sun  Ta_shade
0       0.000000    1     0  5.255013  4.851625
1       0.041667    1    60  5.235818  6.323018
2       0.083333    1   120  4.971423  6.425505
3       0.125000    1   180  4.688973  6.391484
4       0.166667    1   240  4.393941  6.320929
...          ...  ...   ...       ...       ...
8755  364.791667  365  1140  7.273749  8.430885
8756  364.833333  365  1200  6.554797  7.831616
8757  364.875000  365  1260  6.044455  7.337254
8758  364.916667  365  1320  5.385937  6.930348
8759  364.958333  365  1380  5.262752  6.606110

[8760 rows x 5 columns]


In [ ]:
#### incorporating input maps made in R GRASS
# data to incorporate include:
    # net solar radiation (W m-2)
    # slope (unit radians or degrees) ###Slope can be expressed either in degrees or as a percentage. Aspect is the orientation of slope, measured clockwise in degrees from 0 to 360, where 0 is north-facing, 90 is east-facing, 180 is south-facing, and 270 is west-facing.
    # aspect (unit degrees or radians) #grass returns as degrees
    # shade 
    # vegetation 
    # shade view factor = ratio or percent of thermal (longwave) radiation coming in
    
    

In [ ]:

################################ set constants  ######################

#earth essentricity 
e=0.01675 

#So = solar constant = 1360 W m-2
S0=1360 #Wm-2

#boltzman constant, sigma = 5.67e-8
sigma=5.67e-8 # W m-2 k-4

#ordinal day of the year
julian_days= arange(1,366,1)

# eg = ground emissivity
eg=0.75


time_list_day=arange(0,24,0.1)
#julian=4
latitude=33.4484
longitude=112.0740
altitude=350
tau=0.7
rg=0.4 #ground reflectance #when modeled with sears April2019 we used rg=0.4
h=0.1 #SVL of 100mm
d=0.1 #animal diameter in unit meters 

Tave=25.
amp=15.
z=0.
D=0.03
u=0.1 #u=wind in m/s

max_t_yesterday=32.2
min_t_today=17.6
max_t_today=32.2
min_t_tomorrow=17.6


# alpha_s = shortwave rad absorptance of the animal 
alpha_s = 0.8

# alpha_l = longwave rad absorptance of the animal
alpha_l=0.95

#cp = specific heat of air at constant temp 
cp=29.3 #unit = J/(mol*C)

mass=20


##### constants that differ by location ####
tc_latitude = 31.84989
tc_longitude = 109.3257
tc_altitude = 201.


In [ ]:
# ############################### define functions  #####################

# #dd2 = d over d sqared.. correction factor for eliptical orbit of earth at a given J, julian calendar day
# #hSo = amount of direct solar radiation reaching earths surface = So*(dd2**2)

# def solar_rad(julian):
#     return S0*(1.+2.*e*math.cos(((2*math.pi)/365.)*julian))
    
# def f_var(J):
#     return 279.575 + (0.9856*J)

# #ET = equation of time    
# def ET(J):
#     f=f_var(J) ### then you wouldn't need to call the function each time
#     return (((-104.7*math.sin(radians(f)))+(596.2*math.sin(radians(2*f)))+(4.3*math.sin(radians(3*f)))-(12.7*math.sin(radians(4*f)))-(429.3*math.cos(radians(f)))-(2*math.cos(radians(2*f)))+(19.3*math.cos(radians(3*f))))/3600)

# #LC = longitudinal correction
# def LC(long):
#     return (long % 15.)/15.

# #t0 = solar noon
# def t0(J,long):
#     return 12. - ET(J) + LC(long)

# #h = hour angle, the degrees of rotation needed to rotate the earth to a specific number of hours
# def hour(time,J,long):
#     return 15.*(time-t0(J,long))

# #δ = declination angle, angle between the equator and the sun'r rays. Depends only on time of year, not time of day.
# def declination(J):  
#     #solar declination ranges from +23.45 degrees at summer solstice to -23.45 at winter solstics
#     return degrees(math.asin(0.39785*math.sin(radians(278.87+0.9856*J+1.9165*math.sin(radians(356.6+0.9856*J))))))

# z = zenith angle
# def zenith(lat,J,time,long):     #Lambert's cos law says the intensity of radiation is proportional to the cos of the angle between the suns rays and the normal to the irridiated surface
#     #declin=declination(J)
#     #h=hour(time,long,J)
#     #return math.acos(math.sin(radians(lat))*math.sin(radians(declin))+math.cos(radians(lat))*math.cos(radians(declin))*math.cos(radians(h)))
#     return math.acos(math.sin(radians(lat))*math.sin(radians(declination(J)))+ math.cos(radians(lat))*math.cos(radians(declination(J)))*math.cos(radians(hour(time,J,long))))

# #solar rad adjusted by the cosine of zenith angle
# def hS0(J,lat,time,long):
#     return solar_rad(J) * math.cos(zenith(lat,J,time,long))

# #Up to now we have direct radiation hitting atmosphere.. Beer's law accounts for the atmosphere. Longer distance through the atm to the ground, less radiation. Why radiation is more intense at high elevation

# #p_a = the air pressure
# def pa(alt): #altitude (a) is in meters
#     return 101.3*math.exp((-alt)/8200)

# m = optical air mass number
# def air_mass(alt,lat,time,long,J):
#     z=zenith(lat,J,time,long)
#     p_a=pa(alt)
#     return p_a/(101.3*math.cos(z))

# #correcting for radiation hitting the atmosphere to get DIRECT incident solar radiation, hS
def hS(J,lat,time,long,tau,alt):  #Tau = optical transmittance of the atmosphere. 
    #T varies as function of the clarity of the atmosphere. T is 0.75 on a clear day. More typically between 0.6-0.7 on clear day. 
    #Values < 0.4 represent cloudy days
    m=air_mass(alt,lat,time,long,J)
    return solar_rad(J)*(math.cos(zenith(lat,J,time,long)))*(tau**m)


Sd = diffuse shortwave radiation... solar rad that's scatted from reflecting off clouds, particulate matter, atc. 
def Sd(J,lat,time,long,tau,alt):
    return hS0(J,lat,time,long) * 0.3*(1-tau**(degrees(air_mass(alt,lat,time,long,J))))



## Won't need any of the functions above because we're getting shortwave radiation (direct and diffuse) at each time from GRASS Input table



In [ ]:
############################### define functions  #####################


## Won't need any of the functions above because we're getting shortwave radiation (direct and diffuse) at each time from GRASS Input table


# Sr = reflected shortwave radiation
def Sr(rg,J,lat,time,long,tau,alt):
    return rg*(hS(J,lat,time,long,tau,alt)+ Sd(J,lat,time,long,tau,alt))

#Thermal radiation (longwave)
# Sla = longwave radiation from the atmosphere, Sla only varies with air temp 
def Sla(Ta):
    return 53.1*(10**-14)*(Ta+273.15)**6

# Slg = longwave radiation emitted from ground ##OUT
def Slg(eg,Tg):  #ground emissivity
    return eg*sigma*(Tg+273.15)**4

# Longwave radiation heating ground from above (going IN), uses sky view input from GRASS
def longwave_rad_IN(Sview, Ta,thermal_rad_veg_above):
    return (Sview * Sla(Ta))+ ((1-Sview)* thermal_rad_veg_above)

# Net Longwave Radiation = IN - OUT 
def Net_longwave(Sview, Ta,thermal_rad_veg_above,eg,Tg):
    longwave_IN = longwave_rad_IN(Sview, Ta,thermal_rad_veg_above)
    longwave_OUT = Slg(eg,Tg)
    return longwave_IN - longwave_OUT

#Gamma_t = dimensionless diurnal temp function
def Gamma_t(t):
    return 0.44-0.46*math.sin((math.pi/12)*t+0.9) + 0.11*math.sin(2*(math.pi/12)*t + 0.9)

def anytime_temp(t,max_t_yesterday,min_t_today,max_t_today,min_t_tomorrow):
    #t = time
    #i = today
    #i-1 = yesterday
    #i+1 = tomorrow
    #Tn = daily min
    #Tx = daily max
    if 0.<=t<=5.:
        return max_t_yesterday*Gamma_t(t) + min_t_today*(1-Gamma_t(t))
    elif 5.<t<=14.:
        return max_t_today*Gamma_t(t)+ min_t_today*(1-Gamma_t(t))
    else:
        return max_t_today*Gamma_t(t)+ min_t_tomorrow*(1-Gamma_t(t))
    
#statistically predicting Tg at a given depth (z) and time of day (t)
#Tg = ground temperatures
def Tzt(Tave,amp,z,D,t):
    #Tave = average daily soil temp
    #A(0) = amplitude (half of the peak to peak variation) of temp fluctuations at the suface
    #D = damping depth (D=0.1 for moist mineral soils and 0.03-006 for dry mineral soils)
    return Tave+amp*math.exp(-z/D)*math.sin((math.pi/12)*(t-8)-z/D)

# Fh = view factor .....## This is same as sky view factor, correct???
def Fh(h,lat,J,time,long,d):
    zenith_angle=zenith(lat,J,time,long)
    return(1+((4*h*math.sin(zenith_angle))/(math.pi*d)))/(4+((4*h)/d))

# Rabs = amount of radiation absorbed by an organism    
# def Rabs(s,alpha_s,h,lat,J,time,long,d,tau,alt,rg,alpha_l,max_t_yesterday,min_t_today,max_t_today,min_t_tomorrow,eg,Tave,amp,z,D):
#     if math.cos(zenith(latitude,julian,time,longitude)) > 0.:
#         return s*alpha_s*(Fh(h,lat,J,time,long,d)*(hS(J,lat,time,long,tau,alt))+0.5*Sd(J,lat,time,long,tau,alt)+0.5*Sr(rg,J,lat,time,long,tau,alt)) + 0.5*alpha_l*(Sla(anytime_temp(time,max_t_yesterday,min_t_today,max_t_today,min_t_tomorrow)) + Slg(eg,Tzt(Tave,amp,z,D,time))) 
#     else:
#         return 0.5*alpha_l*(Sla(anytime_temp(time,max_t_yesterday,min_t_today,max_t_today,min_t_tomorrow)) + Slg(eg,Tzt(Tave,amp,z,D,time))) 


# function below was modified to calculate Rabs for 1 temp input        
# def Rabs(s,alpha_s,h,lat,J,time,long,d,tau,alt,rg,alpha_l,Ta,eg,Tave,amp,z,D):
#     if math.cos(zenith(latitude,julian,time,longitude)) > 0.:
#         return s*alpha_s*(Fh(h,lat,J,time,long,d)*(hS(J,lat,time,long,tau,alt))+0.5*Sd(J,lat,time,long,tau,alt)+0.5*Sr(rg,J,lat,time,long,tau,alt)) + 0.5*alpha_l*(Sla(anytime_temp(time,max_t_yesterday,min_t_today,max_t_today,min_t_tomorrow)) + Slg(eg,Tzt(Tave,amp,z,D,time))) 
#     else:
#         return 0.5*alpha_l*(Sla(Ta)) + Slg(eg,Tzt(Tave,amp,z,D,time))     



### New Rabs function to incoporate GRASS input table
def Rabs()

    
# gha = boundary layer conductance for heat assuming d = diameter see Table 7.5 campbell and norman
def gha(u,d):
    #u = wind speed (unit m/s)    
    return 1.4*(0.135*math.sqrt(u/d))

# gr = radiative conductance of the animal
def gr(sigma,Ta,cp): #Ta needs to be air temp at height of animal
    return ((4*sigma)*((Ta+273.15)**3))/cp

def operative(Ta,Rabs,eg,sigma,cp,u,d):
    return Ta + ((Rabs-eg*sigma*Ta**4)/cp*(gr(sigma,Ta,cp)+gha(u,d)))

#calculating body temperatures p 202
Tb1=[25]
def Tb2(Te,Tb1,time_at_temp,time_constant):
    return Te + ((Tb1-Te)*(math.exp(-time_at_temp/time_constant)))



                         

In [ ]:
time_list = np.arange(0, 24, 0.1).tolist()
print(type(time_list))

new_time = time_list * 37




Sla_list =[]
for val in Ta_shade:
    Sla_val = Sla(val)
    Sla_list.append(Sla_val)
#plot(Ta_shade,Sla_list)
#ylabel('longwave radiation')
#xlabel('downscaled Ta from NicheMapR')


#what am I trying to do?
### print Rabs list as long as Ta_shade list

#how long is Ta_shade list?
### hourly Ta ... 365 days * 24 hours in day


Rabs_vals_sun1=[]
Rabs_vals_shade1=[]
for val,time,julian in zip(Ta_shade,new_time,DOY):
    Sla_val= Sla(val)
    Slg_val= Slg(eg,Tzt(Tave,amp,z,D,time))
    if math.cos(zenith(latitude,julian,time,longitude)) > 0.:
        S_direct= hS(julian,latitude,time,longitude,tau,altitude)
        S_diffuse= Sd(julian,latitude,time,longitude,tau,altitude)
        S_reflected= Sr(rg,julian,latitude,time,longitude,tau,altitude)
        s=1. #full sun
    else:
        S_direct=0.
        S_diffuse=0.
        S_reflected=0.
        s=0. #full shade
    Rabs_vals_sun1.append(Rabs(s,alpha_s,h,latitude,julian,time,longitude,d,tau,altitude,rg,alpha_l,val,eg,Tave,amp,z,D))   
    Rabs_vals_shade1.append(Rabs(0.,alpha_s,h,latitude,julian,time,longitude,d,tau,altitude,rg,alpha_l,val,eg,Tave,amp,z,D))   
        
DOY_cut,Rabs_cut=[],[]
        
for j,r in zip(DOY,Rabs_vals_sun1):
    if 40<j<60:
        Rabs_cut.append(r)
        DOY_cut.append(j)
        
plot(DOY_cut,Rabs_cut)

#plot(DOY,Rabs_vals_sun1)
title('annual absorbed radiation for animal in full sun - Tempe 2019 ')
ylabel('Rabs (W/m2)')
xlabel('day of year')
#xlim(1,365)
     
    
    
#240
#8760




In [ ]:

operative_sun1,operative_shade1=[],[]
for julian,time,val in zip(DOY,new_time,Ta_shade):
    Rabs_sun= Rabs(1.,alpha_s,h,latitude,julian,time,longitude,d,tau,altitude,rg,alpha_l,val,eg,Tave,amp,z,D)
    operative_sun1.append(operative(val,Rabs_sun,eg,sigma,cp,u,d))
    Rabs_shade=Rabs(0.,alpha_s,h,latitude,julian,time,longitude,d,tau,altitude,rg,alpha_l,val,eg,Tave,amp,z,D)
    operative_shade1.append(operative(val,Rabs_shade,eg,sigma,cp,u,d))
    
plot(dates,operative_sun1,label='Te full sun')
plot(dates,operative_shade1,label='Te shade')
legend()
#title('Operative temperatures for 100mm SVL lizard in Tempe, AZ')
ylabel('Temperature C')
xlabel('day of year')
#ylim(20,45)


# for j,r in zip(DOY,Rabs_vals_sun1):
#     if 40<j<60:
#         Rabs_cut.append(r)
#         DOY_cut.append(j)



In [ ]:
#trying stack overflow method 25Feb

from collections import defaultdict

daily_temps = defaultdict(list)

for day, value in zip(DOY,Ta_sun):
    daily_temps[day].append(value)
    
ranges = dict()
for day, value in daily_temps.items():
    ranges[day] = (min(value), max(value))

    
#print(ranges)

# mins=[]
# for key,value in ranges.items():
#     mins.append(key,value)
    
# print(mins)
    

# for key, value in test_dict.items(): 
#     print (key, value) 

#([(k, test_dict[k]) for k in test_dict]) 


#just use a list comprehension. for example to get the mins you can do [item['min_temp'] for item in mins_maxes] 
#and so forth for the other columns. you could also not convert to a dict by removing the to_dict['records']. 
#you can simply do mins_maxes.min_temp.values. also remove the brackets around the groupby statement 


###### second option from stack overflow



test_df = pd.DataFrame({'DOY': DOY, 'TIME': TIME, 'Ta_sun': Ta_sun})


mins_maxes=test_df.groupby('DOY').agg( \
    min_temp=('Ta_sun', 'min'),
    max_temp=('Ta_sun', 'max')) \
  .reset_index()\
  .to_dict('records')

cat=[item['min_temp'] for item in mins_maxes]
print(cat)

#mins_maxes.min_temp.values

#to print just that column
#print(test_df['Ta_sun'])

# code above is equivalent to code below:
#min_max_df=test_df.groupby('DOY').agg(min_temp=('Ta_sun', 'min'),max_temp=('Ta_sun', 'max')).reset_index().to_dict('records')

